# Error Utilities

> Utilities for converting structured errors to FastHTML responses, alerts, and error pages

In [ ]:
#| default_exp core.errors

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
from typing import Optional, Any
from fasthtml.common import *
from fasthtml.svg import Svg, Path as SvgPath

from cjm_fasthtml_daisyui.components.feedback.alert import alert, alert_colors
from cjm_fasthtml_daisyui.utilities.semantic_colors import stroke_dui, text_dui
from cjm_fasthtml_tailwind.utilities.effects import shadow
from cjm_fasthtml_tailwind.utilities.spacing import m, p
from cjm_fasthtml_tailwind.utilities.sizing import w, h
from cjm_fasthtml_tailwind.utilities.typography import font_size, font_weight, text_align
from cjm_fasthtml_tailwind.utilities.flexbox_and_grid import flex_display, flex_direction, items, justify
from cjm_fasthtml_tailwind.utilities.backgrounds import bg
from cjm_fasthtml_tailwind.utilities.borders import rounded
from cjm_fasthtml_tailwind.core.base import combine_classes

from cjm_fasthtml_app_core.core.html_ids import AppHtmlIds
from cjm_fasthtml_app_core.components.alerts import create_error_alert, create_warning_alert

# Optional: Import error handling library if available
try:
    from cjm_error_handling.core.base import ErrorContext, ErrorSeverity, BaseError
    from cjm_error_handling.core.errors import (
        ValidationError,
        ConfigurationError,
        PluginError,
        WorkerError,
        ResourceError
    )
    _has_error_handling = True
except ImportError:
    _has_error_handling = False

## Error to Alert Conversion

Convert structured errors from the `cjm-error-handling` library into FastHTML alert components.

In [ ]:
#| export
def error_to_alert(
    error: Any,  # Error object (BaseError from cjm-error-handling or standard Exception)
    include_debug_info: bool = False  # Whether to include debug information in the alert
) -> FT:  # Alert component (error or warning)
    """
    Convert an error to an alert component.
    
    - Uses user-friendly message for display
    - Optionally includes debug information
    - Maps severity to alert type (error/warning)
    - Falls back to standard exception str() for non-structured errors
    
    Example:
        ```python
        try:
            result = manager.load_plugin(plugin_meta)
        except PluginError as e:
            return error_to_alert(e)
        ```
    """
    # Check if it's a structured error from cjm-error-handling
    if _has_error_handling and isinstance(error, BaseError):
        # Get user-friendly message
        message = error.get_user_message()
        
        # Get debug info if requested
        details = None
        if include_debug_info:
            details = error.get_debug_message()
        
        # Map severity to alert type
        if error.severity == ErrorSeverity.WARNING:
            return create_warning_alert(message, details)
        else:
            return create_error_alert(message, details)
    else:
        # Fallback for standard exceptions
        return create_error_alert(str(error))

### Example: Converting Errors to Alerts

In [ ]:
# Example 1: Standard exception
try:
    raise ValueError("Invalid configuration value")
except ValueError as e:
    alert_component = error_to_alert(e)
    print("Standard exception alert created")

# Example 2: Structured error (if library available)
if _has_error_handling:
    try:
        raise PluginError(
            message="Failed to load plugin",
            debug_info="Plugin class not found in module",
            context=ErrorContext(operation="load_plugin"),
            plugin_id="test_plugin"
        )
    except PluginError as e:
        alert_component = error_to_alert(e, include_debug_info=True)
        print("Structured error alert with debug info created")

Standard exception alert created
Structured error alert with debug info created


## HTMX Error Response Utilities

Create HTMX-compatible error responses that can update specific page elements.

In [ ]:
#| export
def error_to_htmx_response(
    error: Any,  # Error object (BaseError or Exception)
    target_id: str = AppHtmlIds.ALERT_CONTAINER,  # ID of element to update with error
    include_debug_info: bool = False  # Whether to include debug information
) -> FT:  # Alert component with correct ID for HTMX targeting
    """
    Create an HTMX-compatible error response.
    
    Returns an alert that HTMX can swap into the target element.
    
    Example:
        ```python
        @app.post("/save-config")
        async def save_config(request):
            try:
                config = await load_config()
                return create_success_alert("Saved!")
            except ConfigurationError as e:
                return error_to_htmx_response(e)
        ```
    """
    # Get the alert component
    alert_component = error_to_alert(error, include_debug_info)
    
    # The alert already has the correct ID from create_error_alert/create_warning_alert
    # If a different target is specified, we need to wrap it
    if target_id != AppHtmlIds.ALERT_CONTAINER:
        # Extract the alert div (first child) and rewrap with new ID
        return Div(
            alert_component.children[0],  # The actual alert div
            id=target_id
        )
    
    return alert_component

In [ ]:
# Example: HTMX error response
if _has_error_handling:
    error = ValidationError(
        message="Invalid form data",
        debug_info="Field 'port' must be between 1-65535",
        context=ErrorContext(operation="validate_form"),
        validation_errors={"port": "Out of range"}
    )
    htmx_response = error_to_htmx_response(error)
    print("HTMX error response created")

HTMX error response created


## Error Page Components

Full-page error displays for critical failures or 404/500 errors.

In [ ]:
#| export
def create_error_page(
    title: str = "Error",  # Page title
    message: str = "An error occurred",  # Main error message
    details: Optional[str] = None,  # Optional details or debug info
    show_home_link: bool = True,  # Whether to show a link back to home
    home_path: str = "/"  # Path for the home link (defaults to root)
) -> FT:  # Div element containing the full error page
    """
    Create a full-page error display.
    
    Useful for critical errors or standard HTTP error pages (404, 500, etc.).
    
    Example:
        ```python
        @app.get("/not-found")
        def not_found():
            return create_error_page(
                title="Page Not Found",
                message="The page you're looking for doesn't exist",
                details="Error 404",
                home_path="/dashboard"
            )
        ```
    """
    content = [
        # Error icon
        Div(
            Svg(
                SvgPath(
                    stroke_linecap="round",
                    stroke_linejoin="round",
                    stroke_width="2",
                    d="M12 9v2m0 4h.01m-6.938 4h13.856c1.54 0 2.502-1.667 1.732-3L13.732 4c-.77-1.333-2.694-1.333-3.464 0L3.34 16c-.77 1.333.192 3 1.732 3z"
                ),
                xmlns="http://www.w3.org/2000/svg",
                fill="none",
                viewBox="0 0 24 24",
                cls=combine_classes(w(24), h(24), stroke_dui.error)
            ),
            cls=combine_classes(m.b(6))
        ),
        # Title
        H1(
            title,
            cls=combine_classes(
                font_size._3xl,
                font_weight.bold,
                text_dui.base_content,
                m.b(4)
            )
        ),
        # Message
        P(
            message,
            cls=combine_classes(
                font_size.lg,
                text_dui.base_content.opacity(70),
                m.b(2)
            )
        )
    ]
    
    # Add details if provided
    if details:
        content.append(
            P(
                details,
                cls=combine_classes(
                    font_size.sm,
                    text_dui.base_content.opacity(50),
                    m.b(8)
                )
            )
        )
    else:
        content.append(Div(cls=m.b(8)))  # Spacer
    
    # Add home link if requested
    if show_home_link:
        from cjm_fasthtml_daisyui.components.actions.button import btn, btn_colors
        content.append(
            A(
                "Go to Home",
                href=home_path,
                cls=combine_classes(btn, btn_colors.primary)
            )
        )
    
    return Div(
        *content,
        cls=combine_classes(
            flex_display,
            flex_direction.col,
            items.center,
            justify.center,
            text_align.center,
            p(8),
            m.y.auto
        )
    )

In [ ]:
# Example: Create a 404 error page
create_error_page(
    title="Page Not Found",
    message="The page you're looking for doesn't exist",
    details="Error 404"
)

```html
<div class="flex flex-col items-center justify-center text-center p-8 my-auto">
  <div class="mb-6">
<svg xmlns="http://www.w3.org/2000/svg" viewbox="0 0 24 24" fill="none" class="w-24 h-24 stroke-error"><path d="M12 9v2m0 4h.01m-6.938 4h13.856c1.54 0 2.502-1.667 1.732-3L13.732 4c-.77-1.333-2.694-1.333-3.464 0L3.34 16c-.77 1.333.192 3 1.732 3z" stroke-width="2" stroke-linecap="round" stroke-linejoin="round"></path></svg>  </div>
  <h1 class="text-3xl font-bold text-base-content mb-4">Page Not Found</h1>
  <p class="text-lg text-base-content/70 mb-2">The page you're looking for doesn't exist</p>
  <p class="text-sm text-base-content/50 mb-8">Error 404</p>
<a href="/" class="btn btn-primary">Go to Home</a></div>

```

In [ ]:
#| export
def error_to_page(
    error: Any,  # Error object (BaseError or Exception)
    include_debug_info: bool = False,  # Whether to include debug information
    show_home_link: bool = True,  # Whether to show a link back to home
    home_path: str = "/"  # Path for the home link (defaults to root)
) -> FT:  # Full error page component
    """
    Convert an error to a full-page error display.
    
    Useful for critical errors that need a dedicated page.
    
    Example:
        ```python
        try:
            critical_operation()
        except CriticalError as e:
            return error_to_page(e, include_debug_info=True, home_path="/dashboard")
        ```
    """
    # Check if it's a structured error
    if _has_error_handling and isinstance(error, BaseError):
        message = error.get_user_message()
        details = error.get_debug_message() if include_debug_info else None
        
        # Use severity to determine title
        if error.severity == ErrorSeverity.CRITICAL:
            title = "Critical Error"
        elif error.severity == ErrorSeverity.ERROR:
            title = "Error"
        else:
            title = "Warning"
    else:
        # Fallback for standard exceptions
        title = "Error"
        message = str(error)
        details = None
    
    return create_error_page(
        title=title,
        message=message,
        details=details,
        show_home_link=show_home_link,
        home_path=home_path
    )

In [ ]:
# Example: Convert error to page
if _has_error_handling:
    error = WorkerError(
        message="Worker process crashed",
        debug_info="Worker PID 12345 terminated unexpectedly",
        context=ErrorContext(operation="worker_monitor", worker_pid=12345),
        worker_type="transcription",
        severity=ErrorSeverity.CRITICAL
    )
    error_page = error_to_page(error, include_debug_info=True)
    print("Error page created")

Error page created


In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()